In [1]:
from IPython.display import clear_output
import SALib
from SALib.sample import saltelli
from mesa.batchrunner import BatchRunner
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from model import *
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint
import time
%matplotlib inline

In [2]:
problem = {
    'num_vars': 8,
    'names': ['international_trade', 'max_eff', 'eutax','neighbor_influence','tax_influence',
              'member_trade_multiplier','randomness','eu_strategy'],
    'bounds': [[0, 1], [0.01, 0.1], [5, 30], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]
}

#international_trade, max_eff, eutax, neighbor_influence,
#tax_influence, member_trade_multiplier, randomness, eu_strategy):
# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 2
max_steps = 1
distinct_samples = 1

'''
"other_count":"other_count",
            "average_cooperativeness":"average_cooperativeness",
            "other_wealth":"other_wealth",
            "total_wealth":"total_wealth",
            "member_wealth":"member_wealth",
            "other_eff":"other_eff",
            "total_eff":"total_eff",
            "member_eff":"member_eff"
'''
# Set the outputs
model_reporters = {"Average cooperativeness": lambda m: m.datacollector.get_model_vars_dataframe()["average_cooperativeness"].values[1:],
                   "EU count": lambda m: m.datacollector.get_model_vars_dataframe()["member_count"].values[1:],
                   "Non-EU wealth": lambda m: m.datacollector.get_model_vars_dataframe()["other_wealth"].values[1:],
                   "Total wealth": lambda m: m.datacollector.get_model_vars_dataframe()["total_wealth"].values[1:],
                   "EU wealth": lambda m: m.datacollector.get_model_vars_dataframe()["member_wealth"].values[1:],
                   "Non-EU efficiency": lambda m: m.datacollector.get_model_vars_dataframe()["other_eff"].values[1:],
                   "Total efficiency": lambda m: m.datacollector.get_model_vars_dataframe()["total_eff"].values[1:],
                   "EU-efficiency": lambda m: m.datacollector.get_model_vars_dataframe()["member_eff"].values[1:],
                   "Gini coefficient": lambda m: m.datacollector.get_model_vars_dataframe()["G"].values[1:],
                  }

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)

# READ NOTE BELOW CODE
batch = BatchRunner(RegionModel, 
                    max_steps=max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['international_trade', 'max_eff', 'eutax','neighbor_influence','tax_influence',
                                         'member_trade_multiplier','randomness','eu_strategy'])
                                    #'average_cooperativeness', 'member_count', 'other_wealth', 'total_wealth' , 'member_wealth', 
                                    #    'other_eff','total_eff' ,'member_eff' ])
data['Run'], data['average_cooperativeness'], data['member_count'], data['other_wealth'], data['total_wealth'], data['member_wealth'] , data['other_eff'], data['total_eff'], data['member_eff'], data['G'] = None, None, None, None, None, None, None,None, None, None
#data['Run'], data['average_cooperativeness'],  data['member_count'], data['other_wealth'], data['total_wealth'], data['member_wealth'], data['other_eff'] , data['total_eff'], data['member_eff'] = None, None, None, None, None, None,None, None, None

In [3]:
# Change this for extra strategies
for i, param_val in enumerate(param_values):
    param_values[i, 7] =  0 if param_values[i, 7] < 0.5 else 1
    param_values[i, 0] =  0 if param_values[i, 0] < 0.5 else 1
for i in range(replicates):
    for j, vals in enumerate(param_values):
        vals = list(vals)
        vals[2] = int(vals[2])
        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val

        batch.run_iteration(variable_parameters, tuple(vals), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count # Don't know what causes this, but iteration number is not correctly filled
        data.iloc[count, 0:8] = vals
        data.iloc[count, 8:18] = iteration_data
        count += 1
        time.sleep(5)
        clear_output()
        
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')

11.11% done
1.939291208001076e-05
[1.01853601 1.01035619 1.00916745 1.0057386  1.00476774 1.01775635
 1.02066425 1.01716595 1.01740301 1.0150058  1.01201998 1.00861207
 1.01092419 1.01535763 1.01263106 1.00886412 1.00690491 1.002172
 1.00493808 1.01709444 1.00635678 1.00360937 1.01760321 1.01090148
 1.00348839 1.01937717 1.0177108  1.01286569 1.00683866 1.01585448
 1.01224168 1.01726012 1.0155948  1.0136001  1.01921025 1.01712142
 1.00344505 1.00996627 1.01082225 1.00124495 1.00833195 1.01674271
 1.01055775 1.00753633 1.00521272 1.01275966 1.00515348 1.01774914
 1.00125702 1.00681241 1.00420807 1.00339971 1.01494552 1.00615371
 1.00985088 1.01407913 1.01082514 1.00563117 1.01676648 1.00564711
 1.00348774 1.01237291 1.01112448 1.00505109 1.01586745 1.00135776
 1.00722285 1.00559204 1.01249303 1.0159693  1.01061888 1.00667531
 1.01021156 1.01759699 1.01484687 1.01005193 1.00794037 1.01282156
 1.01540593 1.01538361 1.00650417 1.01582118 1.00553585 1.01064569
 1.01265606 1.00328753 1.01640

KeyboardInterrupt: 

In [ ]:
data

In [ ]:
Si_sheep = sobol.analyze(problem, data['average_cooperativeness'].values, print_to_console=True)
Si_wolves = sobol.analyze(problem, data['other_wealth'].values, print_to_console=True)
def plot_index(s, params, i, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)

    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')

for Si in (Si_sheep, Si_wolves):
    # First order
    plot_index(Si, problem['names'], '1', 'First order sensitivity')
    plt.show()

    # Second order
    plot_index(Si, problem['names'], '2', 'Second order sensitivity')
    plt.show()

    # Total order
    plot_index(Si, problem['names'], 'T', 'Total order sensitivity')
    plt.show()